# Palmer Penguins Modeling

Import the Palmer Penguins dataset and print out the first few rows.

Suppose we want to predict `bill_depth_mm` using the other variables in the dataset.

**Dummify** all variables that require this.

In [7]:
pip install palmerpenguins

In [9]:
import pandas as pd
import palmerpenguins as p
import numpy as np

In [10]:
penguins = p.load_penguins()
penguins.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN,2007
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007


- dummify varables

In [11]:
from sklearn.preprocessing import OneHotEncoder

In [25]:
penguins = penguins.dropna()
encoder = OneHotEncoder(sparse_output=False)
encoded_data = encoder.fit_transform(penguins[['species', 'island', 'sex']])
encoded_df = pd.DataFrame(encoded_data, columns = encoder.get_feature_names_out(['species', 'island', 'sex'])).reset_index()
encoded_df.loc[0:6, :]

,index,species_Adelie,species_Chinstrap,species_Gentoo,island_Biscoe,island_Dream,island_Torgersen,sex_female,sex_male
0,0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1,1,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,2,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
3,3,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
5,5,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
6,6,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [19]:


pen_dum = pd.concat([penguins, encoded_df], axis = 1)
pen_dum

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year,index,species_Adelie,species_Chinstrap,species_Gentoo,island_Biscoe,island_Dream,island_Torgersen,sex_female,sex_male
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007.0,2.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007.0,4.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,male,2007.0,5.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,178.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
218,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,218.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
256,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,256.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,268.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0


- Convert year to string

Let's use the other variables to predict `bill_depth_mm`. Prepare your data and fit the following models on a training dataset subset of the entire dataset:

* Four different models, each containing a different set of predictor variables

Create a plot like the right plot of Fig 1. in our `Model Validation` chapter with the training and test error plotted for each of your four models.

Which of your models was best?

In [ ]:
# Subsetting
X =
y =